# 1. API를 통해 3개월치 데이터를 불러오고 CSV로 저장
- 기간 : 25-07-01 ~ 25-09-30

In [1]:
# 0. 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive 마운트가 완료되었습니다.")

Mounted at /content/drive
Google Drive 마운트가 완료되었습니다.


In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from google.colab import userdata # userdata는 함수 밖에서 import 해도 안전합니다.

In [3]:
# 에러 메세지 API 유출 방지 함수
def parse_api_error(e: Exception) -> str:
    """
    Exception 객체를 받아, 에러 메시지에 API 키가 포함되어 있으면
    "serviceKey" 부분을 파싱하여 숨깁니다.

    Args:
        e: 발생한 Exception 객체 (예: requests.exceptions.HTTPError)

    Returns:
        API 키가 숨겨진 안전한 에러 메시지 문자열
    """
    # 1. 예외(e)를 문자열로 변환합니다.
    error_message = str(e)

    # 2. '?serviceKey='라는 민감한 문자열이 포함되어 있는지 확인합니다.
    if '?serviceKey=' in error_message:
        # 3. 키가 포함된 부분을 잘라내고 대체 텍스트를 추가합니다.
        parsed_message = error_message.split('?serviceKey=')[0] + "?serviceKey=... [KEY HIDDEN]"
        return parsed_message

    # 4. 키가 없으면 원본 메시지를 그대로 반환합니다.
    return error_message

In [4]:
# --- [수정된 부분 1] API 호출 로직을 함수로 분리 ---
def fetch_data_for_day(date_str: str) -> list:
    """
    특정 날짜의 '외국인' 방문자 데이터를 API에서 가져옵니다.
    API 키는 이 함수 내부에서 '지역 변수'로만 존재합니다.
    """

    # 1. API 키를 '지역 변수(key_inside_function)'로 가져옵니다.
    #    이 변수는 변수 탐색기에 표시되지 않고, 함수 종료 시 메모리에서 사라집니다.
    try:
        key_inside_function = userdata.get('Data_go_kr_API_key')
        if not key_inside_function:
            print(f"[{date_str}] Colab 비밀에서 Data_go_kr_API_key를 찾을 수 없습니다.")
            return [] # 빈 리스트 반환
    except Exception as e:
        print(f"[{date_str}] Colab 비밀 로딩 실패: {e}")
        return []

    BASE_URL = "http://apis.data.go.kr/B551011/DataLabService/locgoRegnVisitrDDList"

    # 2. '지역 변수'를 사용하여 params 딕셔너리를 만듭니다.
    #    이 'params' 딕셔너리 역시 '지역 변수'입니다.
    params = {
        "serviceKey": key_inside_function,
        "MobileOS": "ETC",
        "MobileApp": "ProjectA-ML",
        "startYmd": date_str,
        "endYmd": date_str,
        "numOfRows": 1000,
        "pageNo": 1,
        "_type": "json"
    }

    daily_foreign_visitors = [] # 이 날짜에 수집된 데이터를 담을 임시 리스트

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data['response']['header']['resultCode'] != '0000':
            print(f"[{date_str}] API Error: {data['response']['header']['resultMsg']}")
            return [] # 에러 시 빈 리스트 반환

        # (이전의 방어 코드와 동일)
        body = data['response'].get('body')
        if not body or 'items' not in body:
            print(f"[{date_str}] 데이터 없음 (Body 또는 Items 키가 존재하지 않습니다.)")
            return []

        items_data = body['items']
        if not isinstance(items_data, dict) or 'item' not in items_data or not items_data['item']:
            print(f"[{date_str}] 데이터 없음 (Items가 비어있거나(str) item 키가 없습니다.)")
            return []

        items = items_data['item']

        if isinstance(items, dict):
            items = [items]

        # (필터링 로직)
        for item in items:
            if item.get('touDivNm') == '외국인(c)':
                daily_foreign_visitors.append({
                    "date": date_str,
                    "district_code": item.get('signguCode'),
                    "district_name": item.get('signguNm'),
                    "visitor_count": float(item.get('touNum'))
                })

        print(f"[{date_str}] 수집 완료 (외국인 데이터 {len(daily_foreign_visitors)}건)")

    except requests.exceptions.HTTPError as e:
        safe_error_msg = parse_api_error(e) # 이전에 정의한 에러 파싱 함수
        print(f"[{date_str}] HTTP Error: {safe_error_msg}")
    except Exception as e:
        print(f"[{date_str}] Unknown Error: {e}")

    # 3. 함수가 종료되면 'key_inside_function'과 'params' 변수는 사라집니다.
    return daily_foreign_visitors



In [5]:
# --- [수정된 부분 2] 메인 로직은 함수를 호출만 하도록 변경 ---

# 1. 기간 설정 (데이터가 있는 과거 3개월)
start_date = datetime(2025, 7, 1)
end_date = datetime(2025, 9, 30)

# 2. 수집한 모든 데이터를 저장할 '전역 리스트'
#    (이 리스트에는 키가 포함되지 않으므로 안전합니다.)
all_foreign_visitors = []

print(f"데이터 수집을 시작합니다. (기간: {start_date.date()} ~ {end_date.date()})")

current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y%m%d")

    # 3. 위에서 정의한 '안전한' 함수를 호출합니다.
    #    API 키는 이 함수 내부에서만 사용되고 사라집니다.
    results_for_day = fetch_data_for_day(date_str)

    # 4. 결과(키 미포함)만 전역 리스트에 추가합니다.
    if results_for_day:
        all_foreign_visitors.extend(results_for_day)

    current_date += timedelta(days=1)
    time.sleep(0.1)

print("---" * 10)
print(f"총 {len(all_foreign_visitors)}건의 '외국인' 방문 데이터를 수집했습니다.")



데이터 수집을 시작합니다. (기간: 2025-07-01 ~ 2025-09-30)
[20250701] 수집 완료 (외국인 데이터 264건)
[20250702] 수집 완료 (외국인 데이터 264건)
[20250703] 수집 완료 (외국인 데이터 264건)
[20250704] 수집 완료 (외국인 데이터 264건)
[20250705] 수집 완료 (외국인 데이터 264건)
[20250706] 수집 완료 (외국인 데이터 264건)
[20250707] 수집 완료 (외국인 데이터 264건)
[20250708] 수집 완료 (외국인 데이터 264건)
[20250709] 수집 완료 (외국인 데이터 264건)
[20250710] 수집 완료 (외국인 데이터 264건)
[20250711] 수집 완료 (외국인 데이터 264건)
[20250712] 수집 완료 (외국인 데이터 264건)
[20250713] 수집 완료 (외국인 데이터 264건)
[20250714] 수집 완료 (외국인 데이터 264건)
[20250715] 수집 완료 (외국인 데이터 264건)
[20250716] 수집 완료 (외국인 데이터 264건)
[20250717] 수집 완료 (외국인 데이터 264건)
[20250718] 수집 완료 (외국인 데이터 264건)
[20250719] 수집 완료 (외국인 데이터 264건)
[20250720] 수집 완료 (외국인 데이터 264건)
[20250721] 수집 완료 (외국인 데이터 264건)
[20250722] 수집 완료 (외국인 데이터 264건)
[20250723] 수집 완료 (외국인 데이터 264건)
[20250724] 수집 완료 (외국인 데이터 264건)
[20250725] 수집 완료 (외국인 데이터 264건)
[20250726] 수집 완료 (외국인 데이터 264건)
[20250727] 수집 완료 (외국인 데이터 264건)
[20250728] 수집 완료 (외국인 데이터 264건)
[20250729] 수집 완료 (외국인 데이터 264건)
[20250730] 수집 완료 (외국인 데이터 2

In [6]:
# --- 3. Pandas DataFrame 변환 및 로컬 파일 저장 ---
if len(all_foreign_visitors) > 0:
    df = pd.DataFrame(all_foreign_visitors)
    file_name = "foreign_visitors_3months.csv"
    df.to_csv(file_name, index=False, encoding='utf-8-sig')

    print(f"성공: {file_name} 파일이 Colab 로컬 환경에 저장되었습니다.")
    print("\n[저장된 데이터 샘플]")
    print(df.head())
else:
    print("수집된 데이터가 없어 파일을 저장하지 않았습니다.")

성공: foreign_visitors_3months.csv 파일이 Colab 로컬 환경에 저장되었습니다.

[저장된 데이터 샘플]
       date district_code district_name  visitor_count
0  20250701         11110           종로구       24391.06
1  20250701         11140            중구       54452.14
2  20250701         11170           용산구       20624.31
3  20250701         11200           성동구        9331.94
4  20250701         11215           광진구        3323.50


In [7]:
dir = '/content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data'
foreign_data = pd.read_csv(dir)
print(foreign_data)

           date  district_code district_name  visitor_count
0      20250701          11110           종로구       24391.06
1      20250701          11140            중구       54452.14
2      20250701          11170           용산구       20624.31
3      20250701          11200           성동구        9331.94
4      20250701          11215           광진구        3323.50
...         ...            ...           ...            ...
22635  20250930          47850           칠곡군         950.98
22636  20250930          47900           예천군         194.38
22637  20250930          47920           봉화군         217.79
22638  20250930          47930           울진군         581.15
22639  20250930          47940           울릉군        1052.81

[22640 rows x 4 columns]
